<a href="https://colab.research.google.com/github/Alee-Razaa/Alee-Razaa/blob/main/Spam_detection_NLP_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas

In [2]:
import spacy as sp
nlp = sp.load('en_core_web_sm')
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('/content/spam_ham_dataset.csv')

In [27]:
def preprocess_text(text):
    # Tokenize and process the text
    doc = nlp(text)

    # Remove stop words and punctuation, and lemmatize the tokens
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token in ["\n","\r","^"] ]

    # Join the tokens back into a single string
    cleaned_text = ' '.join(tokens)
    return cleaned_text

In [29]:
df['cleaned_text'] = df['text'].apply(preprocess_text)

KeyboardInterrupt: 

In [23]:
df['text'][0]

"Subject: enron methanol ; meter # : 988291\r\nthis is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary\r\nflow data provided by daren } .\r\nplease override pop ' s daily volume { presently zero } to reflect daily\r\nactivity you can obtain from gas control .\r\nthis change is needed asap for economics purposes ."

In [26]:
df['cleaned_text'][:1]

0    subject enron methanol meter 988291 \r\n follo...
Name: cleaned_text, dtype: object

In [30]:
df.shape

(5171, 5)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.cleaned_text, df.label_num, test_size=0.2, random_state=42)
X_train

5132    subject april activity survey \r\n start colle...
2067    subject message subject \r\n hey julie ^ ^ tur...
4716    subject txu fuel sds nomination 2001 \r\n atta...
4710    subject richardson volume nov 99 dec 99 meter ...
2268    subject new era online medical care \r\n new e...
                              ...                        
4426    subject ena sale hpl \r\n legal review contrac...
466     subject tenaska iv \r\n bob \r\n understand sa...
3092    subject broom bristle flew \r\n differentiable...
3772    subject calpine daily gas nomination weekend \...
860     subject meter 1459 6 00 \r\n yep right s june ...
Name: cleaned_text, Length: 4136, dtype: object

In [17]:
X_train.values

5132    subject april activity survey \r\n start colle...
2067    subject message subject \r\n hey julie ^ ^ tur...
4716    subject txu fuel sds nomination 2001 \r\n atta...
4710    subject richardson volume nov 99 dec 99 meter ...
Name: cleaned_text, dtype: object

In [31]:
X_train.values

array(['subject april activity survey \r\n start collect datum april attached survey drive \r\n cost activity commercial team select rcs \r\n fill driver quantity request driver month \r\n february fill attached activity driver survey april \r\n number rc end day thursday 4 th \r\n group currently generate standard report capture \r\n survey datum send report recipient \r\n list begin month eliminate need fill \r\n survey month like collect information \r\n team begin month let know \r\n contact lisa cousino x 3 6343 unable complete \r\n survey allotted time \r\n thank \r\n shari \r\n 3 3859',
       'subject message subject \r\n hey julie ^ ^ turn 18 high school senior houston tx wait long time day finally get wish simply try meet new cyber friend know lead d don t worry usually online free age verification t younger snoop hehe \r\n http oioaz stuntscholar com ju 2 \r\n',
       'subject txu fuel sds nomination 2001 \r\n attach 2001 nomination take gas sale \r\n purchase contract txu 

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X_train)
X_train_count.toarray()[0]

array([0, 0, 0, ..., 0, 0, 0])

In [35]:
X_train_count[0].shape

(1, 41245)

In [39]:
# prompt: Print the non zero values from the "X_train_count.toarray()[0]"

# Get the non-zero values and their indices
non_zero_indices = np.nonzero(X_train_count.toarray()[0])[0]
non_zero_values = X_train_count.toarray()[0][non_zero_indices]

# Print the non-zero values
print(non_zero_values)


[1 1 3 1 3 2 2 1 2 1 1 1 1 1 1 2 1 1 3 1 1 1 3 1 1 1 1 1 1 1 1 4 1 1 1 1 1
 1 2 1 1 1 1 1 1 1 6 2 1 1 1 1 1]


In [42]:
X_test_count = v.transform(X_test)
X_test_count.toarray()[:2]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [43]:
# Step 13: Train a Naive Bayes classifier on the vectorized training data
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_count, y_train)

MultinomialNB()

In [44]:
# Step 14: Predict the labels for the test data
y_pred = model.predict(X_test_count)

In [45]:
# Step 15: Evaluate the model using classification metrics
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       742
           1       0.94      0.95      0.95       293

    accuracy                           0.97      1035
   macro avg       0.96      0.96      0.96      1035
weighted avg       0.97      0.97      0.97      1035



In [52]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [53]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [54]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       742
           1       0.94      0.95      0.95       293

    accuracy                           0.97      1035
   macro avg       0.96      0.96      0.96      1035
weighted avg       0.97      0.97      0.97      1035



In [55]:
# Save the trained pipeline to a file
import joblib
joblib.dump(pipeline, 'spam_classifier_pipeline.pkl')


['spam_classifier_pipeline.pkl']

In [56]:
# Load the model from the file
loaded_pipeline = joblib.load('spam_classifier_pipeline.pkl')


In [65]:
# Define a function to get important words for spam classification
def get_important_words_count_vectorizer(pipeline, class_label=1, top_n=20):
    # Extract the vectorizer and model from the pipeline
    vectorizer = pipeline.named_steps['vectorizer']
    model = pipeline.named_steps['nb']

    # Get feature names and feature importances (log probabilities) from the model
    feature_names = vectorizer.get_feature_names_out()
    class_probabilities = model.feature_log_prob_[class_label]

    # Create a DataFrame to hold feature names and their corresponding probabilities
    feature_probs = pd.DataFrame({
        'Feature': feature_names,
        'Log Probability': class_probabilities
    })

    # Sort by log probability and get top_n features
    top_features = feature_probs.sort_values(by='Log Probability', ascending=False).head(top_n)

    return top_features

In [89]:
def preprocess_text(text):
    doc = nlp(text)
    filtered_text = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(filtered_text)

def get_contributing_words(pipeline, email_text, class_label=1):
    # Preprocess the email text
    preprocessed_text = preprocess_text(email_text)

    # Extract the vectorizer and model from the pipeline
    vectorizer = pipeline.named_steps['vectorizer']
    model = pipeline.named_steps['nb']

    # Transform the email text using the vectorizer
    X_transformed = vectorizer.transform([preprocessed_text])

    # Get feature names
    feature_names = vectorizer.get_feature_names_out()

    # Get the log probabilities for the specific class
    class_probabilities = model.feature_log_prob_[class_label]

    # Get the feature indices for the email text
    feature_indices = X_transformed.indices

    # Get the corresponding feature names and their log probabilities
    feature_probs = {
        feature_names[i]: class_probabilities[i]
        for i in feature_indices
    }

    # Sort features by their log probabilities
    sorted_features = sorted(feature_probs.items(), key=lambda x: x[1], reverse=True)

    return sorted_features


In [71]:
email_text1 = """
The day has come. We’ve made it.  🎉

Your input has helped us understand how to fully round out and shape the future of Lumen5. After many months of hard work and valuable feedback from our amazing community (yes, that includes you!), we are thrilled to announce the our AI Voiceover feature is out of beta.

What does this mean? We have officially unveiled this engaging new type of video and all its features. Here’s a quick rundown:

Choose a stock AI voice from our marketplace

Browse through our selection of human-like AI voices, each with descriptors such as gender, name, accent, and tone/mood. Select the voice that best fits your video’s narrative and audience. Not sure which one to pick? Or what’s the difference between them all? Read more about how to pick the right voice for your video.

Or upload and clone your own (branded) voice

For Enterprise creators, if you have a specific voice associated with your brand, you can upload it for a personalized touch. This ensures consistency and familiarity for your audience. Record and upload a sample recording and let our AI do the rest. The cloned voice can be used for future videos and beyond!

Or create a custom one-of-a-kind AI voice

For a truly unique voice, create a custom AI voice tailored to your brand’s personality. This option offers the highest level of voiceover customization and brand alignment and is offered to Enterprise level teams. If you’re interested, please feel free to book a demo.

Book a demo
Generate and recommend video scenes

Lumen5’s AI will recommend video scenes for you! Generating an entirely new video for you based on your voiceover transcript, this feature will save you tons of time and create consistent results.

Regenerate and swap parts of the voiceover

If any phrases are pronounced incorrectly, you can easily regenerate those parts until you achieve the desired pronunciation. It’s easy, here’s how.

Or if you’re not satisfied with your initial voice choice, swap the voice entirely and experiment with different options until you’re happy with the final output. We’ve outlined it here for you.

Editing the transcript

Inspiration can strike at any moment. Editing multiple words or large chunks of the transcript is now available. Whether you want to tweak the tone, adjust the pacing, or refine the wording, you have the power to perfect your message without starting from scratch. To edit, highlight the section you’d like to edit in the transcript and make your desired changes. And ta-da! The voiceover will automatically regenerate itself to match the transcript.

And that’s everything to it. Looking for a more in-depth breakdown? Here’s our help guide for AI Voiceover Videos.

We hope you like it,

Kaegan and the Lumen5 Product Team
"""


In [74]:
email_text2 = """
Hi Team,

I hope this email finds you well.

I wanted to follow up on the project we discussed last week. I have attached the updated draft of the report, which includes the latest data and analysis. Please review it at your earliest convenience and let me know if there are any additional changes or if you have any questions.

Additionally, could we schedule a brief meeting next week to go over the final details before the presentation? I am available on Tuesday and Thursday afternoon. Please let me know which time works best for you.

Thank you for your attention to this matter. I look forward to your feedback.

Best regards,

Alex Johnson
"""


In [79]:
email_text3 = """Join us Thursday, August 8 at 12:00 PM MT
Join CU Boulder on Thursday, August 8 at noon MT, to learn about their MS in Data Science program. You’ll hear about performance-based admissions, the interdisciplinary curriculum, and career outcomes for this 100% online degree program.

Secure your spot today!


 """

In [92]:
email_text4 = """As Andrew Ng recently noted, "AI agent workflows will drive massive AI progress this year — perhaps even more than the next generation of foundation models."

In line with this pivotal trend, we’ve launched a series of short courses to help you build and implement AI agents through hands-on lessons, while gaining experience with leading frameworks like crewAI, AutoGen, and LangGraph. Many of these are beginner friendly, with just basic Python as a prerequisite recommendation. Check out our available short courses on AI agents!

Multi AI Agent Systems with crewAI

Taught by João Moura, founder and CEO of crewAI, Multi AI Agent Systems with crewAI will teach you key components of AI agents, and how to assign specialized roles to agents and coordinate their efforts for optimal performance.

With crewAI, an open source library for building multi-agent systems, you'll get hands-on experience building agent crews for processes like tailoring resumes and interview prep for job applications, researching, writing, and editing technical articles, conducting customer outreach campaigns and financial analysis, planning events, and more.

You’ll design and prompt a team of agents through natural language to automate your business workflows, exceeding the performance of prompting a single LLM.

Enroll for free

AI Agents in LangGraph

LangGraph allows developers to create highly controllable agents.

In AI Agents in LangGraph you will learn to build an agent from scratch using Python and an LLM, and then you will rebuild it using LangGraph, learning about its components and how to combine them to build flow-based applications.

Additionally, you will learn about agentic search, which returns multiple answers in an agent-friendly format, enhancing the agent’s built-in knowledge. This course will show you how to use agentic search in your applications to provide better data for agents to enhance their output.

Enroll for free

Building Agentic RAG with LlamaIndex

In this course made in collaboration with LlamaIndex and taught by its co-founder and CEO, Jerry Liu, you'll build agents capable of intelligently navigating, summarizing, and comparing information across multiple research papers from arXiv, and learn how to debug these agents, ensuring you can guide their actions effectively.

Unlike the standard retrieval augmented generation (RAG) pipeline—suitable for simple queries across a few documents—agentic RAG adapts based on initial findings to enhance further data retrieval. In Building Agentic RAG with LlamaIndex, you'll use this framework to build research agents skilled in tool use, reasoning, and decision-making with your data.

Enroll for free

AI Agentic Design Patterns with AutoGen

This short course was made in collaboration with Microsoft and Penn State University, and is taught by AutoGen creators Chi Wang, Principal Researcher at Microsoft Research, and Qingyun Wu, Assistant Professor at Penn State University.

In AI Agentic Design Patterns with AutoGen, you’ll learn how to build and customize multi-agent systems, enabling agents to take on different roles and collaborate to accomplish complex tasks using AutoGen, a framework that enables the development of LLM applications using multi-agents, all while implementing four agentic design patterns: Reflection, Tool use, Planning, and Multi-agent collaboration using AutoGen.

Enroll for free

Looking for more? Check out all AI short courses from DeepLearning.AI

Keep learning,

- The DeepLearning.AI team """

In [93]:
# Example email text
email_text = """
Hi there,

I have a Lamborghini that can only go 5 MPH.

That's what it felt knowing my brain moves fast but, because my reading was slow, I couldn't learn fast.

I'd feel frustrated and honestly, felt like a failure.

I remember, when I was 10 years old, I wanted to be the President, a billionaire, and a pop star all at the same time.

But, I realized that to do this… I needed to be able to read and learn fast.

If you keep reading… I'm gonna give you the secrets I learned to learn anything 10x faster…

>>> Click here to join the How to Learn Anything 10x Faster training

I used these secrets to eventually become one of the youngest people on the Forbes 30 Under 30 list and build products that over 40 million people use today.

I'm not saying this to brag, but just to share with you that learning how to learn faster… is worth the effort.

When I was 9 years old, my dad canceled my birthday party.

I was lying to my parents about reading books. I desperately wanted to be like my dad. He was my hero. I wanted him to be proud of me. He cared so much about me reading, but I found it so slow, so boring, so difficult.

Every day, I'd pretend to read. I’d sit with the dead tree in my hands, looking at the page. But, I read so slowly that it was less painful to sit and do nothing than to actually read.

When my dad discovered I lied, he canceled my birthday party.

I cried. Not just because of the party, but because I wasn’t becoming who I wanted to be. My parents thought I was lazy.

I was going to be who I wanted to be. I was going to figure out reading. So for that school break, all I did was read.

No TV, no playing outside, just books. I'd spend hours in the public library, trying to force myself through pages.

I'd fall asleep my nose buried in the 4th page of Harry Potter. The librarian would have to wake me up.

I was determined to brute force my way through. Then, something changed.

My dad started reading to me. He even began recording books on his cassette player for me to listen to. I’d fall asleep listening to his voice.

I learned how to download audiobooks from the internet. Suddenly, I could "read" while brushing my teeth, biking, or walking.

Between the ages of 13 and 30 I averaged reading 2 books a week. More than 2,000 books in total. I got into an Ivy League college, became fluent in English, and learned how to code. Learning fast became my superpower. It completely changed the trajectory of my life.

This is why I'm so passionate about sharing tools like Speechify. I want every person to experience the same transformation I did.

Learning how to learn faster isn't just a skill – it's a life-changer. I'm here to show you how to do it.

I saw that you signed up to use Speechify but never figured out how to use it well. I'd love to teach you how.

This week, I'm putting together a training on "How to Learn Anything 10x Faster" where we'll deep dive into:

- How to Read / Learn 3x Faster (including YouTube videos)
- 1 Secret for Unlocking 1.5x Higher Memory
- 3 ADHD focus secrets for never getting distracted while reading
- How to read PDFs & Kindle eBooks on Speechify
- Using AI to summarize documents
- Why 40% of Billionaires have Dyslexia & how it could be your superpower

I know how it feels to be overwhelmed with all the stuff we need to learn. But what if you could learn things way faster? Think about how much more you could do.

This isn't just about reading books faster. It's about learning anything fast: for work, school, or just things you're interested in.

I've been where you are, and these tricks have made a huge difference for me.

That's why I'm so excited to share them with you.

We can do this together, and I really think this training could help you a lot.

So, want to give it a shot?

Let's learn how to learn faster, together.

I think you'll be amazed at what you can do.

>>> Click here to join the How to Learn Anything 10x Faster training

Much love 💙,
Cliff
"""
# Use the loaded model to make a prediction
prediction = loaded_pipeline.predict([email_text4])

# Interpret the prediction
is_spam = bool(prediction[0])
print("Spam" if is_spam else "Not Spam")

# Predict and evaluate on the test set
y_pred = loaded_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


Spam
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       742
           1       0.94      0.95      0.95       293

    accuracy                           0.97      1035
   macro avg       0.96      0.96      0.96      1035
weighted avg       0.97      0.97      0.97      1035



In [94]:
# Get and print the words contributing to spam classification
contributing_words = get_contributing_words(loaded_pipeline, email_text4)
print("Words contributing to spam classification:")
for word, prob in contributing_words:
    print(f"{word}: {prob:.4f}")

Words contributing to spam classification:
information: -6.0283
look: -6.3693
free: -6.5271
business: -6.6665
like: -6.8592
provide: -6.8961
use: -6.8961
year: -6.9401
microsoft: -6.9457
line: -6.9628
system: -6.9802
base: -7.0784
available: -7.0849
customer: -7.2543
event: -7.2698
performance: -7.3344
source: -7.3427
help: -7.5067
action: -7.5575
process: -7.5575
check: -7.6792
note: -7.7398
well: -7.7524
state: -7.7651
drive: -7.7911
financial: -7.8452
application: -7.9630
experience: -8.0614
lead: -8.0614
natural: -8.0614
design: -8.0788
development: -8.1145
format: -8.1329
gain: -8.1329
build: -8.1516
effort: -8.1900
open: -8.2300
create: -8.2506
allow: -8.3152
team: -8.3152
return: -8.3377
learn: -8.3606
agent: -8.3842
enhance: -8.3842
co: -8.4330
document: -8.4330
compare: -8.4843
datum: -8.4843
search: -8.4843
short: -8.4843
highly: -8.5383
research: -8.5383
tool: -8.5383
key: -8.6253
paper: -8.6879
model: -8.7206
simple: -8.7206
ng: -8.7896
campaign: -8.8260
answer: -8.8637
mul